In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [30]:
%run scripts/helper.py
%run scripts/model_train_plus_test.py

In [3]:
crowd_train = load_file('./data/train.csv/train.csv', index_col='id')
y = crowd_train.median_relevance.values

In [4]:
crowd_test = load_file('./data/test.csv/test.csv', index_col='id')

In [5]:
# Stratified shuffle split
train_idx, test_idx = ssSplit(y, train_size=500, random_state=1234)

In [6]:
# training and test set
Xtrain = crowd_train.iloc[train_idx]
ytrain = y[train_idx]

Xtest = crowd_train.iloc[test_idx]
ytest = y[test_idx]

In [7]:
Xtrain_text = tweak_text(Xtrain)
#Xtest_text = tweak_text(Xtest)
Xtest_text = tweak_text(crowd_test)

In [8]:
# whole corpus
corpus = []

for i in range(len(Xtrain_text)):
    corpus.append(Xtrain_text[i])

for j in range(len(Xtest_text)):
    corpus.append(Xtest_text[j])

In [15]:
Xvalidation_text = tweak_text(Xtest)

In [16]:
for k in range(len(Xvalidation_text)):
    corpus.append(Xvalidation_text[k])

In [17]:
print len(corpus)

24029


In [31]:
Xtrain_fitted, tfv = TFIDF(Xtrain_text, corpus)

In [32]:
# Non linear svm model on processed text
svm, svd, scl = build_non_linear_model(Xtrain_fitted, ytrain)

In [33]:
options = {
    'tfv': tfv,
    'svd': svd,
    'scl': scl
}

In [34]:
svm_pred_non_lin = make_predictions(svm, options, Xvalidation_text)

In [35]:
print 'Kappa score on validation set %0.4f ' %(quadratic_weighted_kappa(ytest, svm_pred_non_lin))

Kappa score on validation set 0.1151 


In [59]:
# Linear svm model on processed text
svm_lin, select = build_linear_model(Xtrain_fitted, ytrain)

In [64]:
options = {
    'tfv': tfv,
    'select': select
}

In [65]:
svm_pred_lin = make_predictions(svm_lin, options, Xtest_text)

In [66]:
print 'Validation set accuracy %0.4f ' %(quadratic_weighted_kappa(ytest, svm_pred_lin))

Validation set accuracy 0.4350 


### Unprocessed text

In [68]:
Xtrain_text_u = prepareText(Xtrain)
Xtest_text_u = prepareText(Xtest)

In [69]:
# whole corpus
corpus_u = []

for i in range(len(Xtrain_text_u)):
    corpus_u.append(Xtrain_text_u[i])

for j in range(len(Xtest_text_u)):
    corpus_u.append(Xtest_text_u[j])

In [77]:
Xtrain_fitted_u, tfv_u = TFIDF(Xtrain_text_u, corpus_u)

In [78]:
# Non linear svm model on unprocessed text
svm_u, svd_u, scl_u = build_non_linear_model(Xtrain_fitted_u, ytrain)

In [79]:
options = {
    'tfv': tfv_u,
    'svd': svd_u,
    'scl': scl_u
}

In [80]:
svm_pred_non_lin_u = make_predictions(svm_u, options, Xtest_text_u)

In [81]:
print 'Validation set accuracy %0.4f ' %(quadratic_weighted_kappa(ytest, svm_pred_non_lin_u))

Validation set accuracy 0.5620 


In [88]:
print 'Ensemble of unprocessed and processed non linear SVM models ', quadratic_weighted_kappa(ytest, (svm_pred_non_lin + svm_pred_non_lin_u) / 2)

Ensemble of unprocessed and processed non linear SVM models  0.605575423197
